In [53]:
import subprocess
from pathlib import Path
import os
import re
from tqdm.notebook import tqdm
import pandas as pd

In [ ]:
label_path = Path("/mnt/h/saranathan_share/ADNI_full/segresnet821_inference")
files = [item.name for item in os.scandir(label_path) if item.is_file()]
subjects = []
for file in files:
    match = re.search(r"(.+)_crop_T1\.nii\.gz", file)
    if match:
        subjects.append(match[1])

In [42]:
all_vols = []
for file, subject in tqdm(zip(files, subjects), total=len(files)):
    cmd = ["fslstats", "-K", label_path/file, label_path/file, "-V"]
    result = subprocess.run(cmd, capture_output=True, text=True)
    result_lines = result.stdout.split("\n")
    vols = []
    for i in range(40):
        try:
            vols.append(float(result_lines[i].split(" ")[1]))
        except (IndexError, ValueError):
            vols.append(None)
    try:
        assert len(vols) == 40
    except AssertionError:
        print(file)
        continue
    else:
        all_vols.append((subject, vols))

  0%|          | 0/387 [00:00<?, ?it/s]

In [60]:
vol_dict = {elem[0]: elem[1] for elem in all_vols}
index = [elem[0] for elem in all_vols]
df = pd.DataFrame.from_dict(vol_dict, orient="index")

new_cols = {}
for i in range(40):
    new_cols[i] = i+1
df = df.rename(columns=new_cols)

df.index.name = "subject"
df.to_csv("ADNI_full_segresnet_volumes.csv")

In [34]:
cmd = ["fslstats", "-K", label_path/file, label_path/file, "-V"]
result = subprocess.run(cmd, capture_output=True, text=True)
result_lines = result.stdout.split("\n")
vols = []
for i in range(40):
    try:
        vols.append(float(result_lines[i].split(" ")[1]))
    except (IndexError, ValueError):
        vols.append(None)
vols
assert len(vols) == 40

In [30]:
vols

[55.199997,
 57.599998,
 217.199982,
 221.999985,
 61.199997,
 61.199997,
 610.799988,
 591.599976,
 226.799988,
 235.199982,
 892.799927,
 938.399963,
 91.199997,
 91.199997,
 29.999998,
 44.399998,
 64.799995,
 68.399994,
 387.599976,
 409.199982,
 13.199999,
 13.199999,
 28.799999,
 23.999998,
 500.399963,
 397.199982,
 2822.399902,
 2931.599854,
 581.999939,
 568.799988,
 326.399994,
 286.799988,
 176.399994,
 163.199997,
 3253.199707,
 3608.399902,
 213.599991,
 220.799988,
 885.599976,
 929.999939]

In [ ]:
cmd = ["fslstats", "-K", label_path/file, label_path/file, "-V"]
result = subprocess.run(cmd, capture_output=True, text=True)
result_lines = result.stdout.split("\n")
vols = []
for line in result_lines:
    try:
        vols.append(float(line.split(" ")[1]))
    except (IndexError, ValueError):
        if "missing" in line:
            vols.append(None)
        else:
            pass

'099_S_4480_crop_T1.nii.gz'